In [8]:
%load_ext autoreload
%autoreload 2
import argparse
import pandas as pd
import os
import torch
from src.utils import get_char, outer_cross_val
import numpy as np
from src.utils import one_hot_encode
from models.data_process import prepare_nonproto_features
from models.dataset import ProtospacerDataset
from torch.utils.data import Dataset, DataLoader
from models.data_process import get_datatensor_partitions, prepare_nonproto_features, generate_partition_datatensor
from models.trainval_workflow import run_inference
from src.utils import compute_eval_results_df
from models.data_process import prepare_the_data_for_user_samples,get_data_ready_for_user_samples 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
cmd_opt = argparse.ArgumentParser(description='Argparser for data')
cmd_opt.add_argument('-data_dir',  type=str, default = './data/', help = 'directory of the data')
cmd_opt.add_argument('-target_dir',  type=str, default='processed',  help = 'folder name to save the processed data')
cmd_opt.add_argument('-working_dir',  type=str, default = './', help = 'the main working directory')
cmd_opt.add_argument('-data_name',  type=str, default='./', help = '')
cmd_opt.add_argument('-feature_list',  type=str, help = 'list of feature names we are gonna consider')
cmd_opt.add_argument('-random_seed', type=int,default=42)
args, _ = cmd_opt.parse_known_args()

In [10]:
args.data_name = 'Endogenous_spacers_TnpB_list.csv'
#args.data_name = 'Endogenous_spacers_TnpB_Nureki_lab.csv'
exp_name = 'protospacer'

In [11]:
data = prepare_the_data_for_user_samples(args)

true target is not provided, the correlation and loss will be meaningless


In [12]:
x_protospacer, y, x_extended_f,x_non_protos_f = data

In [13]:
def compile_prediction(tdir, num_runs=5):
    l = []
    for i in range(num_runs):
        df = pd.read_csv(os.path.join(tdir, f'run_{i}','predictions_test.csv'))
        if 'seq_id' not in df:
            df['seq_id'] = list(range(0, df.shape[0]))
        if 'Unnamed: 0' in df:
            del df['Unnamed: 0']
        df['run_num'] = i
        l.append(df)
    df = pd.concat(l, axis=0, ignore_index=True)
    return df        
    
def compute_avg_predictions(df):
    agg_df = df.groupby(by=['seq_id']).mean()
    agg_df.reset_index(inplace=True)
    for colname in ('run_num', 'Unnamed: 0'):
        if colname in agg_df:
            del agg_df[colname]
    return agg_df

In [14]:
gpu_index = 0
res_desc = {}
num_runs = 5 # number of trained model folds to use for prediction
version=2
for model_name in ['CNN', 'RNN', 'Transformer']:
    args.model_name =  model_name# {'RNN','CNN', 'Transformer'}
    res_desc[model_name] = {}
    for exp_name in ['protospacer']:
        args.exp_name = exp_name
        model_path = os.path.join(args.working_dir, 
                                  'output', 
                                  f'{model_name}_v{version}',
                                  exp_name)
        dpartitions, datatensor_partitions = get_data_ready_for_user_samples(data,
                                                                             args,
                                                                             num_runs=num_runs, # define how many model runs to be used
                                                                             normalize_opt='max',
                                                                             train_size=0., 
                                                                             fdtype=torch.float32)

        train_val_path = os.path.join(model_path, 'train_val')
        test_path = os.path.join(model_path, 'sample_test')
        
        print(f'Running model: {model_name}, exp_name: {exp_name}, saved at {train_val_path}')
        a, b = run_inference(datatensor_partitions, 
                             train_val_path, 
                             test_path, 
                             gpu_index, 
                             to_gpu=True)
                             #num_runs=num_runs)
        print('='*15)
        
        # save all predictions in one dataframe with the corresponding model run
        tdf = compile_prediction(test_path, num_runs=num_runs)
        # compute average prediction across the different runs of the same model
        tdf_ensemble = compute_avg_predictions(tdf)
        tdf.to_csv(os.path.join(test_path, f'{num_runs}fold_predictions.csv'), index=False)
        tdf_ensemble.to_csv(os.path.join(test_path, f'avg_{num_runs}fold_predictions.csv'), index=False)
    
        res_desc[model_name][exp_name] = compute_eval_results_df(test_path, len(dpartitions))        


we are at the testing face
we are at the testing face
we are at the testing face
we are at the testing face
we are at the testing face
we are here
Running model: CNN, exp_name: protospacer, saved at ./output/CNN_v2/protospacer/train_val
cuda:0
test
number of epochs 1
model_name: CNN
input_size: 20
Epoch: 0
Regression report on all events:
MAE:
8.789302917889186
MSE:
138.2582377277839
Correlation coefficient:
nan
Spearman coefficient:
nan
Pearson coefficient:
nan
------------------------------

xxxxxxxxxxxxxxxxxxxxxxxxx
test
number of epochs 1
model_name: CNN
input_size: 20
Epoch: 0
Regression report on all events:
MAE:
8.487133722007275
MSE:
123.44181870552899
Correlation coefficient:
nan
Spearman coefficient:
nan
Pearson coefficient:
nan
------------------------------

xxxxxxxxxxxxxxxxxxxxxxxxx
test
number of epochs 1
model_name: CNN
input_size: 20
Epoch: 0
Regression report on all events:
MAE:
10.819539855207715
MSE:
187.07183866180515
Correlation coefficient:
nan
Spearman coefficien

/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/ho

Epoch: 0
Regression report on all events:
MAE:
9.52442392366273
MSE:
174.47593021072177
Correlation coefficient:
nan
Spearman coefficient:
nan
Pearson coefficient:
nan
------------------------------

xxxxxxxxxxxxxxxxxxxxxxxxx
run_name: run_0
run_name: run_1
run_name: run_2
run_name: run_3
run_name: run_4
we are at the testing face
we are at the testing face
we are at the testing face
we are at the testing face
we are at the testing face
Running model: Transformer, exp_name: protospacer, saved at ./output/Transformer_v2/protospacer/train_val
cuda:0
test
number of epochs 1
model_name: Transformer
input_size: 20
Epoch: 0
Regression report on all events:
MAE:
9.454954287409782
MSE:
200.71785758746697
Correlation coefficient:
nan
Spearman coefficient:
nan
Pearson coefficient:
nan
------------------------------

xxxxxxxxxxxxxxxxxxxxxxxxx
test
number of epochs 1
model_name: Transformer
input_size: 20
Epoch: 0
Regression report on all events:
MAE:
11.275232196279935
MSE:
239.26773361321145
Cor

/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/ubuntu/anaconda3/envs/graphnn/li

Epoch: 0
Regression report on all events:
MAE:
9.089054816109794
MSE:
159.83042636310498
Correlation coefficient:
nan
Spearman coefficient:
nan
Pearson coefficient:
nan
------------------------------

xxxxxxxxxxxxxxxxxxxxxxxxx
run_name: run_0
run_name: run_1
run_name: run_2
run_name: run_3
run_name: run_4


/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
/home/ubuntu/anaconda3/envs/graphnn/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/home/ubuntu/anaconda3/envs/graphnn/li

In [ ]:
'''
gpu_index = 0
res_desc = {}
version=2
for model_name in ['FFN','CNN', 'RNN', 'Transformer']:
    args.model_name =  model_name# {'RNN','CNN', 'Transformer'}
    res_desc[model_name] = {}
    for exp_name in ['protospacer']:
        args.exp_name = exp_name
        model_path = os.path.join(args.working_dir, 
                                  'output', 
                                  f'{model_name}_v{version}',
                                  exp_name)
        dpartitions, datatensor_partitions = get_data_ready_for_user_samples(data,args, 
                                                            normalize_opt='max',
                                                            train_size=0.9, 
                                                            fdtype=torch.float32)

        train_val_path = os.path.join(model_path, 'train_val')
        test_path = os.path.join(model_path, 'sample_test')
        
        print(f'Running model: {model_name}, exp_name: {exp_name}, saved at {train_val_path}')
        a, b = run_inference(datatensor_partitions, 
                             train_val_path, 
                             test_path, 
                             gpu_index, 
                             to_gpu=True)
        print('='*15)
    
        res_desc[model_name][exp_name] = compute_eval_results_df(test_path, len(dpartitions))
        
'''